In [ ]:
import numpy as np
import dedalus.public as d3
import matplotlib.pyplot as plt
import logging
logger = logging.getLogger(__name__)
%config InlineBackend.figure_format = 'retina'

In [ ]:
C = 6**(-1/2)
σ = 1

# most solutions in Toomre, Gough & Speigel use "rigid boundary conditions"
rigid = True

In [ ]:
nz = 256
Lz = 1
dealias = 3/2
dtype = np.float64
coord = d3.Coordinate('z') #d3.CartesianCoordinates(['z'])
dist = d3.Distributor(coord, dtype=dtype)
zbasis = d3.ChebyshevT(coord, size=nz, bounds=(0, Lz), dealias=dealias)

W = dist.Field(name='W', bases=zbasis)
θ = dist.Field(name='θ', bases=zbasis)
T0 = dist.Field(name='T0', bases=zbasis)
τ1 = dist.Field(name='τ1')
τ2 = dist.Field(name='τ2')
τ3 = dist.Field(name='τ3')
τ4 = dist.Field(name='τ4')
τ5 = dist.Field(name='τ5')
τ6 = dist.Field(name='τ6')
τ7 = dist.Field(name='τ7')
τ8 = dist.Field(name='τ8')

R = dist.Field(name='R')

a = dist.Field(name='a')
a['g'] = 1

# Substitutions
dz = lambda A: d3.Differentiate(A, coord)

lift_basis = zbasis.derivative_basis(1)
lift = lambda A, n: d3.Lift(A, lift_basis, n)

D = lambda A: dz(dz(A))-a**2*A

In [ ]:
# Problem
dt = lambda A: 0*A # search for steady solution

problem = d3.EVP([W, θ, τ1, τ2, τ3, τ4, τ5, τ6], eigenvalue=R, namespace=locals())
problem.add_equation("1/σ*dt(D(W)) - D(D(W)) + R*a**2*θ + lift(τ1, -1) + lift(τ2, -2) + lift(τ3, -3) + lift(τ4, -4) = 0")
problem.add_equation("dt(θ) - D(θ) + dz(T0)*W + lift(τ5, -1) + lift(τ6, -2) = 0")

problem.add_equation("θ(z=0) = 0")
problem.add_equation("W(z=0) = 0")
problem.add_equation("θ(z=Lz) = 0")
problem.add_equation("W(z=Lz) = 0")
if rigid:
    problem.add_equation("dz(W)(z=0) = 0")
    problem.add_equation("dz(W)(z=Lz) = 0")
else:
    # stress-free
    problem.add_equation("dz(dz(W))(z=0) = 0")
    problem.add_equation("dz(dz(W))(z=Lz) = 0")

In [ ]:
z = dist.local_grid(zbasis)
T0['g'] = Lz - z

In [ ]:
solver = problem.build_solver()

In [ ]:
for system in ['subsystems', 'solvers']:
    logging.getLogger(system).setLevel(logging.WARNING)

solver.solve_dense(solver.subproblems[0])
eigenvalues = np.sort(solver.eigenvalues.real)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(np.arange(nz//16-1)+1,eigenvalues[1:nz//16])
ax.set_yscale('log')

Searching for the Critical Rayleigh number
-------------------------------------------------

First we search for the critical Rayleigh number $R$ on a discrete $a$ grid.  We use geomspace to create a log-spaced grid, since we're not sure where the minimum lies, and log-spaced grids are generally an efficient way to quickly sample a space.

In [ ]:
a_set = np.geomspace(0.5, 10, num=10)
crit_ra = []
for a_i in a_set:
    a['g'] = a_i
    solver = problem.build_solver()
    solver.solve_dense(solver.subproblems[0])
    eigenvalues = np.sort(solver.eigenvalues.real)
    crit_ra.append(eigenvalues[np.argmin(np.abs(eigenvalues))])

ra_min_discrete = np.min(crit_ra)
a_min_discrete = a_set[np.argmin(crit_ra)]

Having found the minimal critical Ra on the discrete grid, we now use a continuous minimizer (from `scipy.optimize`) to search for a higher accuracy critical Ra.

In [ ]:
from scipy.optimize import minimize

# stub function to assign new a, compute Ra, return min(Ra)
def crit_ra_finder(a_i):
    a['g'] = a_i
    solver = problem.build_solver()
    solver.solve_dense(solver.subproblems[0])
    eigenvalues = np.sort(solver.eigenvalues.real)
    return eigenvalues[np.argmin(np.abs(eigenvalues))]

# this runs the minimization
crit_ra_minimizer = minimize(crit_ra_finder, a_min_discrete)

ra_min = crit_ra_minimizer.fun
a_min = crit_ra_minimizer.x[0]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(a_set, crit_ra)
ax.plot(a_set, crit_ra)
ax.scatter(a_min_discrete, ra_min_discrete, label='discrete min',zorder=5)
ax.scatter(a_min, ra_min, marker='*', label='min', zorder=5, s=100, c='xkcd:dark red')
ax.set_xlabel('a')
ax.set_ylabel('Ra')
ax.legend()

Now, let's compare our critical Ra determined from the discrete grid and from the optimized search (the minimizer) with the literature values from Chandrasekhar.

In [ ]:
ra_Chandra=1707.762
a_Chandra=3.117

print("discrete solution:  Ra_min = {:10.8g} at a = {:6.4g}".format(ra_min_discrete, a_min_discrete))
print("optimized solution: Ra_min = {:10.8g} at a = {:6.4g}".format(ra_min, a_min))
print("Chandrasekhar:      Ra_min = {:10.8g} at a = {:6.4g}".format(ra_Chandra, a_Chandra))